<a href="https://colab.research.google.com/github/ItishaK/AI_Colab_WS/blob/main/ClassificationIrisDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


In [2]:
CSV_COL_NAMES = ['sepal_len','sepal_width','petal_len','petal_width','species']
SPECIES = ['setosa','versicolor','viriginica']

In [3]:
train_ds_path = tf.keras.utils.get_file("iris_train.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

test_ds_path = tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

573/573 [==============================] - 0s 0us/step


In [4]:
train_df = pd.read_csv(train_ds_path, names=CSV_COL_NAMES, header=0)
test_df = pd.read_csv(test_ds_path, names=CSV_COL_NAMES,header=0)

In [5]:
train_df.head()

,sepal_len,sepal_width,petal_len,petal_width,species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
test_df.head()

,sepal_len,sepal_width,petal_len,petal_width,species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [9]:
pred_train = train_df.pop('species')
pred_test = test_df.pop('species')
train_df.head()

,sepal_len,sepal_width,petal_len,petal_width
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [10]:
pred_train.head()

0    2
1    1
2    2
3    0
4    0
Name: species, dtype: int64

In [11]:
pred_train.shape

(120,)

In [12]:
train_df.shape

(120, 4)

In [26]:
def input_fn(features, labels, training=True, batch_size=256):
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
      ds = ds.shuffle(1000).repeat()

    return ds.batch(batch_size)


In [27]:
feature_cols = []
for key in train_df.keys():
    feature_cols.append(tf.feature_column.numeric_column(key=key))
print(feature_cols)

[NumericColumn(key='sepal_len', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='petal_len', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [28]:
iris_classifier = tf.estimator.DNNClassifier(feature_columns=feature_cols,
                                             hidden_units=[30,10],
                                             n_classes=3)

In [32]:
# x = lambda: print("hello")
# x()

iris_classifier.train(input_fn=lambda: input_fn(train_df, pred_train, training=True),
                      steps=5000)

Instructions for updating:
Use standard file utilities to get mtimes.


In [34]:
result = iris_classifier.evaluate(input_fn=lambda: input_fn(test_df, pred_test, training=False))

In [35]:
print(result)

{'accuracy': 0.93333334, 'average_loss': 0.43419355, 'loss': 0.43419355, 'global_step': 10000}


In [36]:
print(result['accuracy'])

0.93333334


In [43]:
print('max(sepal_len): ',train_df['sepal_len'].max())
print('max(sepal_width): ',train_df['sepal_width'].max())
print('max(petal_width): ',train_df['petal_width'].max())
print('max(petal_len): ',train_df['petal_len'].max())

max(sepal_len):  7.9
max(sepal_width):  4.4
max(petal_width):  2.5
max(petal_len):  6.9


In [52]:
def input_pred_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['sepal_len','sepal_width','petal_len','petal_width']
predict = {}

print("Enter the numeric value as indicated")
for value in features:
    val = input(value + ": ")
    predict[value] = [float(val)]

predictions = iris_classifier.predict(input_fn=lambda: input_pred_fn(predict))

for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    prob = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * prob))

Enter the numeric value as indicated
sepal_len: 6.9
sepal_width: 3.1
petal_len: 5.4
petal_width: 2.1
Prediction is "viriginica" (68.4%)


In [48]:
print(pred_dict)

{'logits': array([ 0.7899051,  1.6662621, -0.6894937], dtype=float32), 'probabilities': array([0.27548912, 0.66176146, 0.06274942], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}


In [49]:
expected_result = ['setosa', 'versicolor', 'virginica']

curr_predictions = {
    'sepal_len': [5.1, 5.9, 6.9],
    'sepal_width': [3.3, 3.0, 3.1],
    'petal_len': [1.7, 4.2, 5.4],
    'petal_width': [0.5, 1.5, 2.1]
}